In [1]:
import pandas as pd

file = "Resources/purchase_data.csv"

file_df = pd.read_csv(file)

In [2]:
# Player Count

In [3]:
# Total Number of Players

player_count = len(file_df["SN"].unique())

player_count_df = pd.DataFrame({"Total Players": [player_count]})

player_count_df

,Total Players
0,576


In [4]:
# Purchasing Analysis (Total)

In [5]:
# Number of Unique Items

item_count = len(file_df["Item ID"].unique())

# Average Purchase Price

average_price = "${:.2f}".format(file_df["Price"].mean())

# Total Number of Purchases

total_purchase = len(file_df["Purchase ID"].unique())

# Total Revenue

revenue = "${:.2f}".format(file_df["Price"].sum())

purchasing_analysis_df = pd.DataFrame({
    "Number of Unique Items": [item_count], 
    "Average Price": [average_price],
    "Number of Purchases": [total_purchase],
    "Total Revenue": [revenue]
})

purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [6]:
# Gender Demographics

In [7]:
# Percentage and Count of Male Players

gender = file_df.groupby(["Gender"])

gender_df = gender.agg({"SN": "nunique"})

gender_df = gender_df.rename(columns = {"SN": "Total Count"})

percent = gender_df["Total Count"] / player_count

gender_df["Percentage of Players"] = percent

gender_df = gender_df.sort_values(["Percentage of Players"], ascending=False)

gender_df["Percentage of Players"] = gender_df["Percentage of Players"].map("{:.2%}".format)

gender_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [8]:
# Purchasing Analysis (Gender)

In [9]:
# Purchase Count
count = gender["Purchase ID"].count()

# Average Purchase Price
average = gender["Price"].mean()

# Total Purchase Value
sum_price = gender["Price"].sum()

# Average Purchase Total per Person by Gender
unique = file_df.groupby(["Gender", "SN"])
sum_per_person = unique["Price"].sum()
average_per_person_df = sum_per_person.groupby(["Gender"])
average_per_person = average_per_person_df.mean()

merge_df = pd.merge(count, average, on="Gender")

merge_df = pd.merge(merge_df, sum_price, on="Gender")

merge_df = pd.merge(merge_df, average_per_person, on="Gender")

merge_df = merge_df.rename(columns = {"Purchase ID": "Purchase Count", "Price_x": "Average Purchase Price", "Price_y": "Total Purchase Value", "Price": "Avg Total Purchase per Person"})

merge_df["Average Purchase Price"] = merge_df["Average Purchase Price"].map("${:.2f}".format)
merge_df["Total Purchase Value"] = merge_df["Total Purchase Value"].map("${:.2f}".format)
merge_df["Avg Total Purchase per Person"] = merge_df["Avg Total Purchase per Person"].map("${:.2f}".format)

merge_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [10]:
# Age Demographics

In [11]:
# The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)

bins = [0, 9, 14, 19, 24, 29, 34, 39, 1000]
age_group = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
file_df["Age Group"] = pd.cut(file_df["Age"], bins, labels=age_group, include_lowest=False)

age = file_df.groupby(["Age Group"])

# Total Count
age_df = age.agg({"SN": "nunique"})
# Percent of Players
age_percent = age_df["SN"] / player_count

age_df["Percentage of Players"] = age_percent

age_df = age_df.rename(columns = {"SN": "Total Count"})

age_df["Percentage of Players"] = age_df["Percentage of Players"].map("{:.2%}".format)

age_df

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [12]:
# Purchase Count
age_count = age["Purchase ID"].count()

# Average Purchase Price
age_average = age["Price"].mean()

# Total Purchase Value
age_sum_price = age["Price"].sum()

age_merge_df = pd.merge(age_count, age_average, on="Age Group")
age_merge_df = pd.merge(age_merge_df, age_sum_price, on="Age Group")
age_merge_df = age_merge_df.rename(columns = {"Purchase ID": "Purchase Count", "Price_x": "Average Purchase Price", "Price_y": "Total Purchase Value"})

# Average Purchase Total per Person by Age Group
age_average_per_person = age_merge_df["Total Purchase Value"]/age_df["Total Count"]
age_merge_df["Avg Total Purchase per Person"] = age_average_per_person

age_merge_df["Average Purchase Price"] = age_merge_df["Average Purchase Price"].map("${:.2f}".format)
age_merge_df["Total Purchase Value"] = age_merge_df["Total Purchase Value"].map("${:.2f}".format)
age_merge_df["Avg Total Purchase per Person"] = age_merge_df["Avg Total Purchase per Person"].map("${:.2f}".format)

age_merge_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [13]:
# Top Spenders

In [14]:
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

# SN
unique_SN_df = file_df.groupby(["SN"])

# Purchase Count
SN_Purchase_count = unique_SN_df["Price"].count()

# Total Purchase Value
SN_Total_Purchase = unique_SN_df["Price"].sum()

# Average Purchase Price
SN_Average_Purchase = unique_SN_df["Price"].mean()

SN_merge_df = pd.merge(SN_Purchase_count, SN_Average_Purchase, on="SN")
SN_merge_df = pd.merge(SN_merge_df, SN_Total_Purchase, on="SN")

SN_merge_df = SN_merge_df.rename(columns = {"Price_x": "Purchase Count", "Price_y": "Average Purchase Price", "Price": "Total Purchase Value"})

SN_merge_df = SN_merge_df.sort_values(["Total Purchase Value"], ascending=False)

SN_merge_df["Average Purchase Price"] = SN_merge_df["Average Purchase Price"].map("${:.2f}".format)
SN_merge_df["Total Purchase Value"] = SN_merge_df["Total Purchase Value"].map("${:.2f}".format)

SN_merge_df.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [15]:
# Most Popular Items

In [16]:
# Identify the 5 most popular items by purchase count, then list (in a table):

# Item ID, # Item Name
item_df = file_df.groupby(["Item ID", "Item Name"])

# Purchase Count
item_df_count = item_df["Purchase ID"].count()

# Item Price
item_df_price = item_df["Price"].mean()

# Total Purchase Value
item_df_purchase_total = item_df["Price"].sum()

item_merge_df = pd.merge(item_df_count, item_df_price, on=["Item ID", "Item Name"])
item_merge_df = pd.merge(item_merge_df, item_df_purchase_total, on=["Item ID", "Item Name"])

item_merge_df = item_merge_df.rename(columns = {"Purchase ID": "Purchase Count", "Price_x": "Item Price", "Price_y": "Total Purchase Value"})

item_merge_df["Item Price"] = item_merge_df["Item Price"].map("${:.2f}".format)
item_merge_df["Total Purchase Value"] = item_merge_df["Total Purchase Value"].map("${:.2f}".format)

item_merge_df = item_merge_df.sort_values(["Purchase Count"], ascending=False)

item_merge_df.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [17]:
# Most Profitable Items

In [18]:
# Identify the 5 most profitable items by total purchase value, then list (in a table):

# Item ID, # Item Name
item_df = file_df.groupby(["Item ID", "Item Name"])

# Purchase Count
item_df_count = item_df["Purchase ID"].count()

# Item Price
item_df_price = item_df["Price"].mean()

# Total Purchase Value
item_df_purchase_total = item_df["Price"].sum()

item_merge_df = pd.merge(item_df_count, item_df_price, on=["Item ID", "Item Name"])
item_merge_df = pd.merge(item_merge_df, item_df_purchase_total, on=["Item ID", "Item Name"])

item_merge_df = item_merge_df.rename(columns = {"Purchase ID": "Purchase Count", "Price_x": "Item Price", "Price_y": "Total Purchase Value"})

item_merge_df = item_merge_df.sort_values(["Total Purchase Value"], ascending=False)

item_merge_df["Item Price"] = item_merge_df["Item Price"].map("${:.2f}".format)
item_merge_df["Total Purchase Value"] = item_merge_df["Total Purchase Value"].map("${:.2f}".format)

item_merge_df.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
